In [11]:
!pip install albumentations tqdm matplotlib


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import torch
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

ROOT_DIR = "/workspace/"
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_GEN = "/workspace/SRGAN/gen.pth.tar"
CHECKPOINT_DISC = "/workspace/SRGAN/disc.pth.tar"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-4
NUM_EPOCHS = 1000
BATCH_SIZE = 16
NUM_WORKERS = 4
HIGH_RES = 96
LOW_RES = HIGH_RES // 4
IMG_CHANNELS = 3


highres_transform = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2(),
    ]
)

lowres_transform = A.Compose(
    [
        A.Resize(width=LOW_RES, height=LOW_RES, interpolation=Image.BICUBIC),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
        ToTensorV2(),
    ]
)

both_transforms = A.Compose(
    [
        A.RandomCrop(width=HIGH_RES, height=HIGH_RES),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
    ]
)

test_transform = A.Compose(
    [
        A.RandomCrop(width=LOW_RES, height=LOW_RES),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
        ToTensorV2(),
    ]
)

In [53]:
import torch.nn as nn
from torchvision.models import vgg19
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.utils import save_image


class MyImageFolder(Dataset):
    def __init__(self, root_dir):
        super(MyImageFolder, self).__init__()
        self.data = []
        self.root_dir = root_dir
        self.image_files = [file for file in os.listdir(root_dir) if file.endswith(('png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_files[index])
        try:
            image = np.array(Image.open(img_path))  # Ensure 3-channel
        except OSError:
            print(f"[Corrupted] Index {index}: {self.image_files[index]}")
            # Return a dummy black image of size 128x128 with 3 channels
            image = np.zeros((128, 128, 3), dtype=np.uint8)
        image = both_transforms(image=image)["image"]
        high_res = highres_transform(image=image)["image"]
        low_res = lowres_transform(image=image)["image"]
        return low_res, high_res


def test():
    dataset = MyImageFolder(root_dir="/workspace/SRGAN/data/DIV2K_train_HR/")
    loader = DataLoader(dataset, batch_size=1, num_workers=8)

    for idx, (low_res, high_res) in enumerate(loader):
        print(low_res.shape)
        print(high_res.shape)

# test()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def plot_examples(low_res_folder, gen):
    files = [file for file in os.listdir(low_res_folder) if file.endswith(('png'))]
    print(f"Plotting examples. Length file is {len(files)}")
    gen.eval()
    for file in files:
        image = Image.open(low_res_folder + file)
         
         # Apply test_transform
        transformed = test_transform(image=np.asarray(image))["image"]  # shape [C,H,W], tensor

        # Save the transformed image
        save_image(transformed, f"/workspace/SRGAN/data/test_cropped/{file}")

        # Run generator on transformed image
        with torch.no_grad():
            upscaled_img = gen(transformed.unsqueeze(0).to(DEVICE))
        save_image(upscaled_img * 0.5 + 0.5, f"/workspace/SRGAN/data/saved/{file}")
    gen.train()

# VSS Loss
# phi_5,4 5th conv layer before maxpooling but after activation

class VGGLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg = vgg19(pretrained=True).features[:36].eval().to(DEVICE)
        self.loss = nn.MSELoss()

        for param in self.vgg.parameters():
            param.requires_grad = False

    def forward(self, input, target):
        vgg_input_features = self.vgg(input)
        vgg_target_features = self.vgg(target)
        return self.loss(vgg_input_features, vgg_target_features)


# Modelling

class ConvBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        discriminator=False,
        use_act=True,
        use_bn=True,
        **kwargs,
    ):
        super().__init__()
        self.use_act = use_act
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs, bias=not use_bn)
        self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
        self.act = (
            nn.LeakyReLU(0.2, inplace=True)
            if discriminator
            else nn.PReLU(num_parameters=out_channels)
        )

    def forward(self, x):
        return self.act(self.bn(self.cnn(x))) if self.use_act else self.bn(self.cnn(x))


class UpsampleBlock(nn.Module):
    def __init__(self, in_c, scale_factor):
        super().__init__()
        self.conv = nn.Conv2d(in_c, in_c * scale_factor ** 2, 3, 1, 1)
        self.ps = nn.PixelShuffle(scale_factor)  # in_c * 4, H, W --> in_c, H*2, W*2
        self.act = nn.PReLU(num_parameters=in_c)

    def forward(self, x):
        return self.act(self.ps(self.conv(x)))


class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block1 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.block2 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            use_act=False,
        )

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        return out + x


class Generator(nn.Module):
    def __init__(self, in_channels=3, num_channels=64, num_blocks=16):
        super().__init__()
        self.initial = ConvBlock(in_channels, num_channels, kernel_size=9, stride=1, padding=4, use_bn=False)
        self.residuals = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_blocks)])
        self.convblock = ConvBlock(num_channels, num_channels, kernel_size=3, stride=1, padding=1, use_act=False)
        self.upsamples = nn.Sequential(UpsampleBlock(num_channels, 2), UpsampleBlock(num_channels, 2))
        self.final = nn.Conv2d(num_channels, in_channels, kernel_size=9, stride=1, padding=4)

    def forward(self, x):
        initial = self.initial(x)
        x = self.residuals(initial)
        x = self.convblock(x) + initial
        x = self.upsamples(x)
        return torch.tanh(self.final(x))


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 64, 128, 128, 256, 256, 512, 512]):
        super().__init__()
        blocks = []
        for idx, feature in enumerate(features):
            blocks.append(
                ConvBlock(
                    in_channels,
                    feature,
                    kernel_size=3,
                    stride=1 + idx % 2,
                    padding=1,
                    discriminator=True,
                    use_act=True,
                    use_bn=False if idx == 0 else True,
                )
            )
            in_channels = feature

        self.blocks = nn.Sequential(*blocks)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((6, 6)),
            nn.Flatten(),
            nn.Linear(512*6*6, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1),
        )

    def forward(self, x):
        x = self.blocks(x)
        return self.classifier(x)



In [54]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
torch.backends.cudnn.benchmark = True

def plot_losses(gen_losses, disc_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(gen_losses, label='Generator Loss')
    plt.plot(disc_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Generator and Discriminator Losses Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

def train_fn(loader, disc, gen, opt_gen, opt_disc, mse, bce, vgg_loss, gen_losses, disc_losses):
    loop = tqdm(loader, leave=True)
    gen_epoch_loss = 0.0
    disc_epoch_loss = 0.0
    num_batches = 0

    
    for idx, (low_res, high_res) in enumerate(loop):
        high_res = high_res.to(DEVICE)
        low_res = low_res.to(DEVICE)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        fake = gen(low_res)
        disc_real = disc(high_res)
        disc_fake = disc(fake.detach())
        disc_loss_real = bce(
            disc_real, torch.ones_like(disc_real) - 0.1 * torch.rand_like(disc_real)
        )
        disc_loss_fake = bce(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = disc_loss_fake + disc_loss_real

        opt_disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        # Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        disc_fake = disc(fake)

        adversarial_loss = 1e-3 * bce(disc_fake, torch.ones_like(disc_fake))
        loss_for_vgg = 0.006 * vgg_loss(fake, high_res)
        gen_loss = loss_for_vgg + adversarial_loss

        opt_gen.zero_grad()
        gen_loss.backward()
        opt_gen.step()

        # Record losses
        gen_losses.append(gen_loss.item())
        disc_losses.append(loss_disc.item())
        
        # accumulate batch losses
        gen_epoch_loss += gen_loss.item()
        disc_epoch_loss += loss_disc.item()
        num_batches += 1

    # return average loss for this epoch
    return gen_epoch_loss / num_batches, disc_epoch_loss / num_batches

def main():
    dataset = MyImageFolder(root_dir="/workspace/SRGAN/data/DIV2K_train_HR/")
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        pin_memory=True,
        num_workers=NUM_WORKERS,
    )
    gen = Generator(in_channels=3).to(DEVICE)
    disc = Discriminator(in_channels=3).to(DEVICE)
    opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))
    opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999))
    mse = nn.MSELoss()
    bce = nn.BCEWithLogitsLoss()
    vgg_loss = VGGLoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN,
            gen,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
           CHECKPOINT_DISC, disc, opt_disc, LEARNING_RATE,
        )

    gen_losses = []
    disc_losses = []

    for epoch in range(NUM_EPOCHS):
        gen_loss, disc_loss = train_fn(loader, disc, gen, opt_gen, opt_disc, mse, bce, vgg_loss, gen_losses, disc_losses)
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]  Gen Loss: {gen_loss:.4f}  Disc Loss: {disc_loss:.4f}")
        plot_examples("/workspace/SRGAN/data/test/", gen)
        if SAVE_MODEL:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
            save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)
            
    plot_losses(gen_losses, disc_losses)

In [56]:
main()

=> Loading checkpoint
=> Loading checkpoint


100%|██████████| 50/50 [00:21<00:00,  2.34it/s]


Epoch [1/10000]  Gen Loss: 0.0082  Disc Loss: 0.4862
Plotting examples. Length file is 61
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 50/50 [00:21<00:00,  2.34it/s]


Epoch [2/10000]  Gen Loss: 0.0084  Disc Loss: 0.3606
Plotting examples. Length file is 61
=> Saving checkpoint
=> Saving checkpoint


...

Epoch [157/10000]  Gen Loss: 0.0076  Disc Loss: 0.3412
Plotting examples. Length file is 61
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 50/50 [00:21<00:00,  2.28it/s]


Epoch [158/10000]  Gen Loss: 0.0079  Disc Loss: 0.3470
Plotting examples. Length file is 61
=> Saving checkpoint
=> Saving checkpoint


 16%|█▌        | 8/50 [00:04<00:23,  1.82it/s]


KeyboardInterrupt: 